# 🔬 GPT 시리즈의 진화: GPT-1 → GPT-2 → GPT-3
# Architectural Evolution of the GPT Series

---

## 학습 목표 (Learning Objectives)

이 노트북을 완료하면 다음을 이해할 수 있습니다:

1. GPT-1, GPT-2, GPT-3 사이의 **구체적인 아키텍처 차이점**을 설명할 수 있다
2. **Pre-Norm vs Post-Norm** LayerNorm 배치의 차이와 그 효과를 이해한다
3. **Weight initialization scaling** ($1/\sqrt{N}$)의 필요성과 원리를 이해한다
4. GPT-3의 **Sparse Attention** 패턴을 이해하고 dense attention과 비교할 수 있다
5. **Zero-shot → Few-shot → In-context learning**으로의 패러다임 전환을 실험적으로 확인한다
6. HuggingFace를 활용하여 GPT 모델의 내부 구조를 직접 탐색할 수 있다

---

### 📚 핵심 논문 레퍼런스 (Key Paper References)

| Model | Paper | Authors | Year |
|-------|-------|---------|------|
| GPT-1 | *Improving Language Understanding by Generative Pre-Training* | Radford et al. | 2018 |
| GPT-2 | *Language Models are Unsupervised Multitask Learners* | Radford et al. | 2019 |
| GPT-3 | *Language Models are Few-Shot Learners* | Brown et al. | 2020 |
| Transformer | *Attention Is All You Need* | Vaswani et al. | 2017 |
| Sparse Transformer | *Generating Long Sequences with Sparse Transformers* | Child et al. | 2019 |

---
## Part 0: 환경 설정 (Environment Setup)

In [ ]:
# 필요한 라이브러리 설치
!pip install -q transformers torch matplotlib numpy pandas seaborn

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import pandas as pd
from transformers import (
    GPT2LMHeadModel, GPT2Tokenizer, GPT2Config,
    OpenAIGPTLMHeadModel, OpenAIGPTTokenizer, OpenAIGPTConfig,
    pipeline
)
import warnings
warnings.filterwarnings('ignore')

# 시각화 스타일
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

print("✅ 모든 라이브러리가 성공적으로 로드되었습니다!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

---
## Part 1: GPT 시리즈 전체 개요 (Overview)

GPT(Generative Pre-trained Transformer) 시리즈는 Vaswani et al. (2017)의 Transformer 아키텍처 중 **Decoder 부분**만을 사용하는 autoregressive language model입니다.

세 모델 모두 기본적으로 같은 원리를 공유합니다:
- **Decoder-only** Transformer
- **Causal (masked) self-attention**: 각 토큰은 이전 토큰들만 참조
- **Next-token prediction** 목적함수
- **Learned positional embeddings** (sinusoidal이 아닌 학습된 위치 임베딩)

그러나 세대를 거듭하면서 **"단순히 크기만 키운 것"이 아닌** 중요한 아키텍처 변경과 패러다임 전환이 있었습니다.

In [ ]:
# GPT 시리즈 핵심 스펙 비교표
comparison_data = {
    '항목': [
        'Parameters', 'Layers', 'd_model', 'Attention Heads',
        'd_head', 'd_ff', 'Context Length', 'Vocab Size',
        'Training Data', 'Training Data Size',
        'Training Paradigm', 'LayerNorm Position',
        'Activation Function', 'Attention Type'
    ],
    'GPT-1 (2018)': [
        '117M', '12', '768', '12',
        '64', '3072', '512', '40,478',
        'BooksCorpus', '~5GB (약 10억 단어)',
        'Pre-train + Fine-tune', 'Post-Norm (기존 Transformer 방식)',
        'GELU', 'Dense (Full)'
    ],
    'GPT-2 (2019)': [
        '1.5B (최대)', '48 (최대)', '1600 (최대)', '25 (최대)',
        '64', '6400 (최대)', '1024', '50,257',
        'WebText', '~40GB (약 80억 단어)',
        'Zero-shot (Fine-tune 없이)', 'Pre-Norm ✨',
        'GELU', 'Dense (Full)'
    ],
    'GPT-3 (2020)': [
        '175B', '96', '12,288', '96',
        '128', '49,152', '2048', '50,257',
        'Common Crawl + WebText2 + Books + Wikipedia', '~570GB (약 3000억 토큰)',
        'Few-shot / In-context Learning ✨', 'Pre-Norm',
        'GELU', 'Alternating Dense + Sparse ✨'
    ]
}

df = pd.DataFrame(comparison_data)
df = df.set_index('항목')

# 스타일 적용하여 출력
def highlight_changes(val):
    if '✨' in str(val):
        return 'background-color: #fff3cd; font-weight: bold'
    return ''

styled_df = df.style.applymap(highlight_changes)
styled_df

> **✨ 표시**는 이전 세대 대비 새롭게 도입된 핵심 변경사항을 나타냅니다.
>
> 단순히 파라미터 수를 늘린 것이 아니라, LayerNorm 위치, 학습 패러다임, Attention 구조 등에서
> 중요한 아키텍처 혁신이 있었음을 주목하세요.

---
## Part 2: GPT-1 아키텍처 깊이 보기

### 2.1 GPT-1의 핵심 특징

Radford et al. (2018)의 GPT-1은 다음과 같은 핵심 아이디어를 제시했습니다:

1. **Semi-supervised learning**: 비지도 사전학습(unsupervised pre-training) → 지도 미세조정(supervised fine-tuning)
2. **Decoder-only Transformer**: Encoder-Decoder Attention 제거, Masked Self-Attention만 사용
3. **Task-specific input transformations**: 각 downstream task에 맞는 입력 변환 방식 정의

#### 아키텍처 세부사항 (원문 인용)

> "*We trained a 12-layer decoder-only transformer with masked self-attention heads
> (768 dimensional states and 12 attention heads). For the position-wise feed-forward networks,
> we used 3072 dimensional inner states.*"  
> — Radford et al. (2018), "Improving Language Understanding by Generative Pre-Training"

#### Post-Norm (기존 Transformer 방식)

GPT-1은 Vaswani et al. (2017)의 원래 Transformer와 동일하게 **Post-Norm**을 사용합니다:

$$\text{Output} = \text{LayerNorm}(x + \text{SubLayer}(x))$$

즉, Residual connection 이후에 LayerNorm을 적용합니다.

In [ ]:
# GPT-1 모델 로드 및 구조 확인
print("=" * 60)
print("GPT-1 모델 로드 중... (openai-community/openai-gpt)")
print("=" * 60)

gpt1_config = OpenAIGPTConfig()
print("\n📋 GPT-1 Configuration:")
print(f"  - Vocab size: {gpt1_config.vocab_size}")
print(f"  - Embedding dim (n_embd): {gpt1_config.n_embd}")
print(f"  - Num layers (n_layer): {gpt1_config.n_layer}")
print(f"  - Num heads (n_head): {gpt1_config.n_head}")
print(f"  - Max position embeddings: {gpt1_config.n_positions}")
print(f"  - Activation function: {gpt1_config.afn}")

In [ ]:
# GPT-1 모델의 실제 구조 확인
gpt1_model = OpenAIGPTLMHeadModel.from_pretrained('openai-community/openai-gpt')

print("\n🏗️ GPT-1 전체 아키텍처:")
print(gpt1_model)

In [ ]:
# GPT-1 파라미터 수 상세 분석
def count_parameters_by_module(model, model_name):
    """모듈별 파라미터 수를 계산하고 표로 출력"""
    param_counts = {}
    for name, param in model.named_parameters():
        # 최상위 모듈 이름 추출
        top_module = name.split('.')[0] + '.' + name.split('.')[1] if '.' in name else name
        if top_module not in param_counts:
            param_counts[top_module] = 0
        param_counts[top_module] += param.numel()

    total = sum(param_counts.values())
    print(f"\n📊 {model_name} 파라미터 분석:")
    print(f"{'Module':<40} {'Parameters':>15} {'Percentage':>10}")
    print("-" * 65)
    for module, count in sorted(param_counts.items(), key=lambda x: -x[1]):
        print(f"{module:<40} {count:>15,} {count/total*100:>9.2f}%")
    print("-" * 65)
    print(f"{'TOTAL':<40} {total:>15,}")
    return total

gpt1_total = count_parameters_by_module(gpt1_model, "GPT-1")

### ✏️ 연습 2.1: GPT-1 단일 Transformer Block 분석

GPT-1의 단일 Transformer block의 파라미터 수를 수동으로 계산해 봅시다.

In [ ]:
# ============================================================
# 연습 2.1: GPT-1의 단일 Transformer Block 파라미터 수 계산
# ============================================================

d_model = 768   # embedding dimension
n_heads = 12    # number of attention heads
d_ff = 3072     # feed-forward hidden dimension

# TODO: 아래 빈칸을 채우세요
# Hint: Multi-Head Attention에는 Q, K, V projection + Output projection이 있습니다
#       각 projection은 (d_model x d_model) weight + d_model bias를 가집니다

# Multi-Head Attention 파라미터
# Q, K, V 각각: weight(d_model x d_model) + bias(d_model)
attn_qkv_params = ___  # TODO: Q + K + V projection 파라미터 수
attn_out_params = ___  # TODO: Output projection 파라미터 수
attn_total = attn_qkv_params + attn_out_params

# Feed-Forward Network 파라미터
# 첫 번째 linear: d_model → d_ff (weight + bias)
# 두 번째 linear: d_ff → d_model (weight + bias)
ffn_params = ___  # TODO: FFN 전체 파라미터 수

# LayerNorm 파라미터 (2개: attention 후 + FFN 후)
# 각 LayerNorm: gamma(d_model) + beta(d_model)
ln_params = ___  # TODO: LayerNorm 파라미터 수

block_total = attn_total + ffn_params + ln_params

print(f"Attention 파라미터: {attn_total:,}")
print(f"FFN 파라미터:       {ffn_params:,}")
print(f"LayerNorm 파라미터: {ln_params:,}")
print(f"Block 전체:         {block_total:,}")
print(f"12 Blocks 전체:     {block_total * 12:,}")

In [ ]:
# ============================================================
# 정답 확인: 실제 모델에서 추출하여 비교
# ============================================================

block0 = gpt1_model.transformer.h[0]
block_params_actual = sum(p.numel() for p in block0.parameters())
print(f"\n✅ 실제 GPT-1 Block 0 파라미터 수: {block_params_actual:,}")

# 각 서브모듈 확인
for name, module in block0.named_children():
    params = sum(p.numel() for p in module.parameters())
    print(f"  {name}: {params:,}")

---
## Part 3: GPT-1 → GPT-2 핵심 변경사항

GPT-2 (Radford et al., 2019)는 단순히 모델을 키운 것이 아닙니다. 다음과 같은 **아키텍처 수준의 변경**이 있었습니다:

### 🔑 변경사항 1: Pre-Norm (LayerNorm 위치 변경)

> "*Layer normalization was moved to the input of each sub-block, similar to a pre-activation residual network,
> and an additional layer normalization was added after the final self-attention block.*"  
> — Radford et al. (2019), "Language Models are Unsupervised Multitask Learners"

**Post-Norm (GPT-1):**
$$\text{Output} = \text{LayerNorm}(x + \text{SubLayer}(x))$$

**Pre-Norm (GPT-2):**
$$\text{Output} = x + \text{SubLayer}(\text{LayerNorm}(x))$$

Pre-Norm은 gradient flow를 안정화하여 더 깊은 네트워크의 학습을 가능하게 합니다.
이 변경 덕분에 GPT-2는 최대 48 layers, GPT-3는 96 layers까지 확장될 수 있었습니다.

### 🔑 변경사항 2: 잔차 경로의 Weight Initialization Scaling

> "*We scale the weights of residual layers at initialization by a factor of $1/\sqrt{N}$
> where N is the number of residual layers.*"  
> — Radford et al. (2019)

Residual connection이 누적되면서 발생하는 분산(variance) 증가를 방지합니다.
각 Transformer block에는 2개의 residual connection이 있으므로, $N = 2 \times n_{\text{layers}}$입니다.

### 🔑 변경사항 3: Vocabulary & Context 확장

- Vocabulary: 40,478 → **50,257** (Byte-level BPE)
- Context window: 512 → **1,024** 토큰
- Batch size: 64 → **512**

### 🔑 변경사항 4: 패러다임 전환 — Zero-shot Task Transfer

GPT-1: Pre-train → **Fine-tune** (각 task별 추가 학습 필요)  
GPT-2: Pre-train → **Zero-shot** (추가 학습 없이 바로 task 수행)

### 3.1 실습: Pre-Norm vs Post-Norm 시각화 및 구현

In [ ]:
# ============================================================
# Pre-Norm vs Post-Norm Transformer Block 구현
# ============================================================

class PostNormBlock(nn.Module):
    """GPT-1 스타일: Post-Norm Transformer Block
    Output = LayerNorm(x + SubLayer(x))
    """
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None):
        # Attention + Residual → LayerNorm
        attn_out, _ = self.attn(x, x, x, attn_mask=attn_mask)
        x = self.ln1(x + self.dropout(attn_out))  # Post-Norm: LN after residual

        # FFN + Residual → LayerNorm
        ffn_out = self.ffn(x)
        x = self.ln2(x + ffn_out)  # Post-Norm: LN after residual
        return x


class PreNormBlock(nn.Module):
    """GPT-2 스타일: Pre-Norm Transformer Block
    Output = x + SubLayer(LayerNorm(x))
    """
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_mask=None):
        # LayerNorm → Attention → Residual
        normed = self.ln1(x)  # Pre-Norm: LN before sublayer
        attn_out, _ = self.attn(normed, normed, normed, attn_mask=attn_mask)
        x = x + self.dropout(attn_out)

        # LayerNorm → FFN → Residual
        normed = self.ln2(x)  # Pre-Norm: LN before sublayer
        ffn_out = self.ffn(normed)
        x = x + ffn_out
        return x


print("Post-Norm Block (GPT-1 style):")
print(PostNormBlock(768, 12, 3072))
print("\n" + "=" * 60)
print("\nPre-Norm Block (GPT-2 style):")
print(PreNormBlock(768, 12, 3072))

In [ ]:
# ============================================================
# Pre-Norm vs Post-Norm에서의 Gradient Flow 비교 실험
# ============================================================

def build_stacked_model(block_class, n_layers, d_model, n_heads, d_ff):
    """여러 층의 Transformer Block을 쌓은 모델 생성"""
    layers = nn.ModuleList([block_class(d_model, n_heads, d_ff) for _ in range(n_layers)])
    return layers

def measure_gradient_norms(block_class, n_layers_list, d_model=256, n_heads=4, d_ff=512):
    """각 층 수에 대해 gradient norm을 측정"""
    results = {}

    for n_layers in n_layers_list:
        torch.manual_seed(42)
        layers = build_stacked_model(block_class, n_layers, d_model, n_heads, d_ff)

        # Forward pass
        x = torch.randn(2, 16, d_model, requires_grad=True)
        h = x
        for layer in layers:
            h = layer(h)

        # Backward pass
        loss = h.sum()
        loss.backward()

        # 각 층의 attention weight gradient norm 기록
        grad_norms = []
        for i, layer in enumerate(layers):
            for name, param in layer.named_parameters():
                if 'in_proj_weight' in name and param.grad is not None:
                    grad_norms.append(param.grad.norm().item())
                    break

        results[n_layers] = grad_norms

    return results

# 실험 실행
layer_counts = [4, 8, 16, 24]

print("Gradient norm 측정 중...")
post_norm_grads = measure_gradient_norms(PostNormBlock, layer_counts)
pre_norm_grads = measure_gradient_norms(PreNormBlock, layer_counts)

# 시각화
fig, axes = plt.subplots(1, len(layer_counts), figsize=(20, 5))
fig.suptitle('Gradient Norms: Post-Norm (GPT-1) vs Pre-Norm (GPT-2)', fontsize=16, fontweight='bold')

for idx, n_layers in enumerate(layer_counts):
    ax = axes[idx]
    layers_range = range(len(post_norm_grads[n_layers]))

    ax.plot(layers_range, post_norm_grads[n_layers], 'r-o', label='Post-Norm (GPT-1)', alpha=0.7)
    ax.plot(layers_range, pre_norm_grads[n_layers], 'b-s', label='Pre-Norm (GPT-2)', alpha=0.7)
    ax.set_title(f'{n_layers} Layers')
    ax.set_xlabel('Layer Index')
    ax.set_ylabel('Gradient Norm')
    ax.legend(fontsize=8)
    ax.set_yscale('log')

plt.tight_layout()
plt.show()

print("\n💡 관찰: Pre-Norm은 깊은 층에서도 gradient가 더 안정적으로 유지됩니다.")
print("   이것이 GPT-2/3가 더 깊은 네트워크를 학습할 수 있었던 핵심 이유 중 하나입니다.")

### 3.2 실습: Weight Initialization Scaling ($1/\sqrt{N}$) 실험

In [ ]:
# ============================================================
# Residual stream의 분산 폭발 문제와 1/√N scaling의 효과
# ============================================================

def simulate_residual_variance(n_layers, use_scaling=False):
    """
    Residual connection을 통과하면서 activations의 분산이
    어떻게 변하는지 시뮬레이션합니다.

    use_scaling=True일 때 GPT-2의 1/√N scaling을 적용합니다.
    """
    d_model = 256
    batch_size = 32
    seq_len = 64
    N = 2 * n_layers  # 각 block에 2개의 residual connection

    torch.manual_seed(42)
    x = torch.randn(batch_size, seq_len, d_model)

    variances = [x.var().item()]

    for i in range(n_layers):
        # Attention sublayer의 output 시뮬레이션
        attn_out = torch.randn_like(x) * 0.02  # 초기 가중치로 인한 작은 출력

        if use_scaling:
            # GPT-2 scaling: 잔차 층의 출력을 1/√N으로 스케일링
            attn_out = attn_out / np.sqrt(N)

        x = x + attn_out  # Residual connection
        variances.append(x.var().item())

        # FFN sublayer의 output 시뮬레이션
        ffn_out = torch.randn_like(x) * 0.02

        if use_scaling:
            ffn_out = ffn_out / np.sqrt(N)

        x = x + ffn_out  # Residual connection
        variances.append(x.var().item())

    return variances

# 48 layers (GPT-2 XL과 동일)에서 비교
n_layers = 48
var_no_scale = simulate_residual_variance(n_layers, use_scaling=False)
var_with_scale = simulate_residual_variance(n_layers, use_scaling=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(var_no_scale, 'r-', linewidth=2, label='Without scaling')
ax1.plot(var_with_scale, 'b-', linewidth=2, label='With 1/√N scaling (GPT-2)')
ax1.set_xlabel('Residual Connection Index')
ax1.set_ylabel('Activation Variance')
ax1.set_title(f'Activation Variance over {n_layers} layers')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 정규화해서 비율 비교
ax2.plot(np.array(var_no_scale) / var_no_scale[0], 'r-', linewidth=2, label='Without scaling')
ax2.plot(np.array(var_with_scale) / var_with_scale[0], 'b-', linewidth=2, label='With 1/√N scaling (GPT-2)')
ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Initial variance (baseline)')
ax2.set_xlabel('Residual Connection Index')
ax2.set_ylabel('Variance Ratio (relative to initial)')
ax2.set_title('Normalized Variance Growth')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 Scaling 없이 {n_layers}층 후 분산 변화: {var_no_scale[0]:.4f} → {var_no_scale[-1]:.4f} ({var_no_scale[-1]/var_no_scale[0]:.2f}x)")
print(f"📉 1/√N Scaling 적용 후 분산 변화: {var_with_scale[0]:.4f} → {var_with_scale[-1]:.4f} ({var_with_scale[-1]/var_with_scale[0]:.2f}x)")
print(f"\n💡 1/√N scaling이 residual stream의 분산을 안정적으로 유지시키는 것을 확인할 수 있습니다.")

### 3.3 실습: 실제 GPT-1 vs GPT-2 모델 구조 비교

In [ ]:
# ============================================================
# GPT-2 모델 로드 및 GPT-1과 구조 비교
# ============================================================

gpt2_config = GPT2Config()  # GPT-2 small (124M) 기본 설정
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

print("📋 GPT-1 vs GPT-2 Configuration 비교:")
print(f"{'항목':<30} {'GPT-1':>15} {'GPT-2 (small)':>15}")
print("=" * 60)
print(f"{'Vocab size':<30} {gpt1_config.vocab_size:>15,} {gpt2_config.vocab_size:>15,}")
print(f"{'Embedding dim':<30} {gpt1_config.n_embd:>15} {gpt2_config.n_embd:>15}")
print(f"{'Num layers':<30} {gpt1_config.n_layer:>15} {gpt2_config.n_layer:>15}")
print(f"{'Num heads':<30} {gpt1_config.n_head:>15} {gpt2_config.n_head:>15}")
print(f"{'Max positions (context)':<30} {gpt1_config.n_positions:>15} {gpt2_config.n_positions:>15}")

gpt1_params = sum(p.numel() for p in gpt1_model.parameters())
gpt2_params = sum(p.numel() for p in gpt2_model.parameters())
print(f"{'Total parameters':<30} {gpt1_params:>15,} {gpt2_params:>15,}")

In [ ]:
# ============================================================
# 핵심 구조 차이 확인: LayerNorm 위치
# ============================================================

print("🔍 GPT-1 Block 0 구조 (Post-Norm):")
print("-" * 40)
for name, _ in gpt1_model.transformer.h[0].named_children():
    print(f"  {name}")

print(f"\n🔍 GPT-2 Block 0 구조 (Pre-Norm):")
print("-" * 40)
for name, _ in gpt2_model.transformer.h[0].named_children():
    print(f"  {name}")

print("\n💡 주목할 점:")
print("  - GPT-2에는 'ln_1', 'ln_2'가 각 sublayer 전에 위치 (Pre-Norm)")
print("  - GPT-2에는 최종 출력 전에 추가 LayerNorm ('ln_f')이 있음:")
print(f"  - GPT-2 final LayerNorm: {gpt2_model.transformer.ln_f}")

### ✏️ 연습 3.1: GPT-2의 Weight Initialization Scaling 확인

GPT-2 논문에 따르면, residual layer의 projection weight는 $1/\sqrt{N}$으로 스케일링됩니다.
실제 HuggingFace 코드에서 이를 확인해봅시다.

In [ ]:
# ============================================================
# 연습 3.1: GPT-2 초기화 시 projection weight의 std 분석
# ============================================================

# 새로운 (사전학습되지 않은) GPT-2 모델을 초기화합니다
fresh_gpt2 = GPT2LMHeadModel(GPT2Config())

# Attention의 c_proj (output projection)와 FFN의 c_proj weight의 std를 확인합니다
attn_proj_stds = []
ffn_proj_stds = []
other_stds = []

for name, param in fresh_gpt2.named_parameters():
    if 'c_proj.weight' in name and 'attn' in name:
        attn_proj_stds.append((name, param.std().item()))
    elif 'c_proj.weight' in name and 'mlp' in name:
        ffn_proj_stds.append((name, param.std().item()))
    elif 'c_attn.weight' in name:
        other_stds.append((name, param.std().item()))

print("📊 GPT-2 Weight Initialization Analysis")
print("=" * 60)

# 이론값 계산
n_layer = GPT2Config().n_layer  # 12
base_std = 0.02
N_residual = 2 * n_layer  # 24
expected_proj_std = base_std / np.sqrt(N_residual)

print(f"\n이론적 기대값:")
print(f"  - 일반 weight std: {base_std}")
print(f"  - Projection weight std: {base_std} / √{N_residual} = {expected_proj_std:.6f}")

print(f"\n실제 Attention c_proj weight std (처음 3개):")
for name, std in attn_proj_stds[:3]:
    print(f"  {name}: {std:.6f}")

print(f"\n실제 FFN c_proj weight std (처음 3개):")
for name, std in ffn_proj_stds[:3]:
    print(f"  {name}: {std:.6f}")

print(f"\n일반 c_attn weight std (처음 3개):")
for name, std in other_stds[:3]:
    print(f"  {name}: {std:.6f}")

# TODO: 아래 질문에 답해보세요
print("\n" + "=" * 60)
print("❓ 질문: c_proj weight의 std가 일반 weight의 std보다 작은 이유는 무엇일까요?")
print("   Hint: Residual connection의 누적 효과를 생각해보세요.")

---
## Part 4: GPT-2 → GPT-3 핵심 변경사항

GPT-3 (Brown et al., 2020)는 GPT-2의 아키텍처를 거의 그대로 사용하면서도 두 가지 핵심 혁신을 도입했습니다:

### 🔑 변경사항 1: Alternating Dense + Sparse Attention

> "*We use the same model and architecture as GPT-2, including the modified initialization,
> pre-normalization, and reversible tokenization described therein, with the exception that
> we use alternating dense and locally banded sparse attention patterns in the layers
> of the transformer, similar to the Sparse Transformer.*"  
> — Brown et al. (2020), "Language Models are Few-Shot Learners"

- **Dense (Full) Attention**: 모든 이전 토큰에 attend (일반적인 방식)
- **Locally Banded Sparse Attention**: 인접한 일정 범위의 토큰에만 attend
- 이 두 패턴을 **번갈아가며** 사용 (Layer 0: Dense, Layer 1: Sparse, Layer 2: Dense, ...)

이를 통해 96 layers의 깊은 모델에서도 computational cost를 관리할 수 있었습니다.

### 🔑 변경사항 2: In-Context Learning 패러다임

GPT-1: Pre-train → **Fine-tune**  
GPT-2: Pre-train → **Zero-shot** (task description만으로 수행)  
GPT-3: Pre-train → **Few-shot / In-context Learning** (몇 개의 예시를 prompt에 포함)

> GPT-3 논문에서는 **Zero-shot**, **One-shot**, **Few-shot** 세 가지 in-context learning 설정을
> 체계적으로 정의하고, fine-tuning 없이도 task를 수행할 수 있음을 보여주었습니다.

### 🔑 변경사항 3: 극단적 스케일링

- 175B parameters (GPT-2 대비 ~117x)
- 96 attention layers
- d_model = 12,288 / 96 heads / d_head = 128
- 학습 데이터: ~300B 토큰 (5개 데이터소스의 가중 합)
- Context window: 2,048 토큰

### 4.1 실습: Dense vs Sparse Attention 시각화

In [ ]:
# ============================================================
# Dense vs Locally-Banded Sparse Attention 패턴 시각화
# ============================================================

def create_causal_mask(seq_len):
    """Standard causal (dense) attention mask"""
    mask = torch.tril(torch.ones(seq_len, seq_len))
    return mask

def create_sparse_banded_mask(seq_len, bandwidth=4):
    """Locally-banded sparse attention mask (GPT-3 style)"""
    causal = torch.tril(torch.ones(seq_len, seq_len))
    band = torch.zeros(seq_len, seq_len)
    for i in range(seq_len):
        start = max(0, i - bandwidth + 1)
        band[i, start:i+1] = 1.0
    return causal * band

seq_len = 16
bandwidth = 4  # 실제 GPT-3에서는 더 큰 window 사용

dense_mask = create_causal_mask(seq_len)
sparse_mask = create_sparse_banded_mask(seq_len, bandwidth)

# GPT-3 스타일: alternating pattern
fig, axes = plt.subplots(1, 4, figsize=(22, 5))

titles = [
    'Layer 0: Dense\n(Full Causal)',
    'Layer 1: Sparse\n(Locally Banded)',
    'Layer 2: Dense\n(Full Causal)',
    'Layer 3: Sparse\n(Locally Banded)'
]

masks = [dense_mask, sparse_mask, dense_mask, sparse_mask]
colors = ['Blues', 'Oranges', 'Blues', 'Oranges']

for idx, (ax, title, mask, cmap) in enumerate(zip(axes, titles, masks, colors)):
    sns.heatmap(mask.numpy(), ax=ax, cmap=cmap, cbar=False,
                square=True, linewidths=0.5, linecolor='white')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('Key Position')
    ax.set_ylabel('Query Position')

plt.suptitle('GPT-3: Alternating Dense & Sparse Attention Patterns',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# 계산량 비교
dense_ops = dense_mask.sum().item()
sparse_ops = sparse_mask.sum().item()
print(f"\n📊 Attention 연산량 비교 (sequence length = {seq_len}):")
print(f"  Dense layer: {int(dense_ops)} attention connections")
print(f"  Sparse layer: {int(sparse_ops)} attention connections")
print(f"  Sparse layer 절약: {(1 - sparse_ops/dense_ops)*100:.1f}%")
print(f"  Alternating 평균 절약: {(1 - (dense_ops+sparse_ops)/(2*dense_ops))*100:.1f}%")

### ✏️ 연습 4.1: Sparse Attention의 복잡도 분석

Dense attention은 $O(n^2)$, Locally-banded sparse attention은 $O(n \cdot w)$ ($w$=window size)의
복잡도를 가집니다. 아래 코드를 완성하여 이를 시각적으로 확인해봅시다.

In [ ]:
# ============================================================
# 연습 4.1: Attention 복잡도 분석
# ============================================================

seq_lengths = [64, 128, 256, 512, 1024, 2048, 4096]
window_size = 256  # 대표적인 sparse attention window size

# TODO: 아래 빈칸을 채우세요
dense_complexity = [___  for n in seq_lengths]     # O(n^2): 각 토큰이 모든 이전 토큰 참조
sparse_complexity = [___ for n in seq_lengths]     # O(n*w): 각 토큰이 window 내 토큰만 참조
alternating_complexity = [___ for n in seq_lengths] # 평균: (dense + sparse) / 2

plt.figure(figsize=(12, 6))
plt.plot(seq_lengths, dense_complexity, 'r-o', linewidth=2, label='Dense Attention: O(n²)')
plt.plot(seq_lengths, sparse_complexity, 'g-s', linewidth=2, label=f'Sparse Attention: O(n·{window_size})')
plt.plot(seq_lengths, alternating_complexity, 'b-^', linewidth=2, label='GPT-3 Alternating (average)')
plt.xlabel('Sequence Length')
plt.ylabel('Number of Attention Operations')
plt.title('Attention Complexity: Dense vs Sparse vs GPT-3 Alternating')
plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.show()

# n=2048 (GPT-3의 context length)에서 비교
n = 2048
print(f"\n📊 n = {n} (GPT-3 context length)에서의 비교:")
print(f"  Dense: {n*n:,} operations")
print(f"  Sparse (w={window_size}): {n*window_size:,} operations")
print(f"  GPT-3 Alternating: {(n*n + n*window_size)//2:,} operations")
print(f"  절약률: {(1 - (n*n + n*window_size)/(2*n*n))*100:.1f}%")

### 4.2 GPT-3의 학습 데이터 구성

In [ ]:
# ============================================================
# GPT-3 Training Data Composition 시각화
# ============================================================
# Source: Brown et al. (2020), Table 2.2

data_sources = {
    'Dataset': ['Common Crawl\n(filtered)', 'WebText2', 'Books1', 'Books2', 'Wikipedia'],
    'Tokens (Billions)': [410, 19, 12, 55, 3],
    'Weight in Training Mix (%)': [60, 22, 8, 8, 3],
    'Epochs': [0.44, 2.9, 1.9, 0.43, 3.4]
}

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# 1. Raw token count
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
axes[0].barh(data_sources['Dataset'], data_sources['Tokens (Billions)'], color=colors, edgecolor='white')
axes[0].set_xlabel('Tokens (Billions)')
axes[0].set_title('Raw Token Count by Source', fontweight='bold')
for i, v in enumerate(data_sources['Tokens (Billions)']):
    axes[0].text(v + 5, i, f'{v}B', va='center', fontweight='bold')

# 2. Training weight
axes[1].pie(data_sources['Weight in Training Mix (%)'],
           labels=data_sources['Dataset'],
           autopct='%1.0f%%',
           colors=colors,
           startangle=90,
           textprops={'fontsize': 10})
axes[1].set_title('Weight in Training Mix', fontweight='bold')

# 3. Epochs per dataset
bars = axes[2].barh(data_sources['Dataset'], data_sources['Epochs'], color=colors, edgecolor='white')
axes[2].axvline(x=1.0, color='red', linestyle='--', alpha=0.5, label='1 epoch')
axes[2].set_xlabel('Epochs')
axes[2].set_title('Epochs per Dataset', fontweight='bold')
axes[2].legend()
for i, v in enumerate(data_sources['Epochs']):
    axes[2].text(v + 0.05, i, f'{v}x', va='center', fontweight='bold')

plt.suptitle('GPT-3 Training Data Composition\n(Brown et al., 2020)',
             fontsize=16, fontweight='bold', y=1.03)
plt.tight_layout()
plt.show()

print("💡 핵심 관찰:")
print("  1. Common Crawl이 전체 토큰의 82%를 차지하지만, 학습 시에는 60%로 다운샘플링")
print("  2. 고품질 데이터(WebText2, Wikipedia)는 2-3 에폭 반복 학습 (오버피팅 허용)")
print("  3. 저품질 대규모 데이터(Common Crawl)는 1 에폭 미만으로 학습")
print("  4. 총 학습 토큰: ~300B (전체 499B 중 가중 샘플링)")

---
## Part 5: In-Context Learning 실험 (GPT-2로 체험)

GPT-3의 핵심 혁신인 **in-context learning**을 GPT-2 모델로 간접 체험해봅니다.

- **Zero-shot**: Task description만 제공
- **One-shot**: Task description + 1개 예시
- **Few-shot**: Task description + 여러 예시

In [ ]:
# ============================================================
# In-Context Learning 실험
# GPT-2를 사용하여 Zero-shot / One-shot / Few-shot 비교
# ============================================================

# GPT-2 medium 로드 (더 큰 모델일수록 in-context learning 능력 향상)
print("GPT-2 모델 로드 중...")
generator = pipeline('text-generation', model='gpt2', device=-1)  # CPU
print("✅ 모델 로드 완료!")

# Sentiment Analysis를 통한 in-context learning 실험
print("\n" + "=" * 70)
print("📝 실험: Sentiment Classification via In-Context Learning")
print("=" * 70)

# Zero-shot
zero_shot_prompt = """Review: This movie was absolutely wonderful and I loved every moment of it.
Sentiment:"""

# One-shot
one_shot_prompt = """Review: I hated this movie, it was terrible.
Sentiment: Negative

Review: This movie was absolutely wonderful and I loved every moment of it.
Sentiment:"""

# Few-shot
few_shot_prompt = """Review: I hated this movie, it was terrible.
Sentiment: Negative

Review: Best film I have seen this year!
Sentiment: Positive

Review: Not worth watching, very boring.
Sentiment: Negative

Review: An outstanding masterpiece of cinema.
Sentiment: Positive

Review: This movie was absolutely wonderful and I loved every moment of it.
Sentiment:"""

prompts = {
    'Zero-shot': zero_shot_prompt,
    'One-shot': one_shot_prompt,
    'Few-shot (4 examples)': few_shot_prompt
}

for setting, prompt in prompts.items():
    print(f"\n{'─' * 50}")
    print(f"🔬 {setting}")
    print(f"{'─' * 50}")

    result = generator(
        prompt,
        max_new_tokens=5,
        num_return_sequences=3,
        temperature=0.7,
        do_sample=True,
        pad_token_id=generator.tokenizer.eos_token_id
    )

    for i, r in enumerate(result):
        generated = r['generated_text'][len(prompt):].strip().split('\n')[0]
        print(f"  Output {i+1}: {generated}")

print("\n" + "=" * 70)
print("💡 관찰:")
print("  - Few-shot에서 'Positive'를 출력할 확률이 높아짐을 확인하세요")
print("  - GPT-2는 GPT-3보다 작아서 in-context learning 능력이 제한적이지만,")
print("    패턴을 따라하려는 경향을 관찰할 수 있습니다")
print("  - GPT-3 (175B)에서는 이 능력이 극적으로 향상되었습니다")

### ✏️ 연습 5.1: 자신만의 In-Context Learning 실험

아래 코드를 수정하여 **번역(Translation)** 또는 **요약(Summarization)** 등
다른 task에 대해 in-context learning을 실험해보세요.

In [ ]:
# ============================================================
# 연습 5.1: 나만의 In-Context Learning 실험
# ============================================================

# TODO: 아래 prompt를 수정하여 다른 task를 실험해보세요
# 예시: 번역, 분류, 변환 등

my_prompt = """
English: Hello, how are you?
Korean: 안녕하세요, 어떻게 지내세요?

English: Thank you very much.
Korean: 정말 감사합니다.

English: I love studying natural language processing.
Korean:"""

result = generator(
    my_prompt,
    max_new_tokens=20,
    num_return_sequences=3,
    temperature=0.7,
    do_sample=True,
    pad_token_id=generator.tokenizer.eos_token_id
)

print("🔬 나의 In-Context Learning 실험 결과:")
for i, r in enumerate(result):
    generated = r['generated_text'][len(my_prompt):].strip().split('\n')[0]
    print(f"  Output {i+1}: {generated}")

---
## Part 6: 전체 파라미터 스케일 비교 시각화

In [ ]:
# ============================================================
# GPT-1/2/3 파라미터 스케일 비교
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(20, 7))

# 1. 파라미터 수 비교 (로그 스케일)
models = ['GPT-1\n(2018)', 'GPT-2 Small\n(2019)', 'GPT-2 Medium', 'GPT-2 Large', 'GPT-2 XL', 'GPT-3\n(2020)']
params = [0.117, 0.124, 0.355, 0.774, 1.5, 175]
colors_bar = ['#3498db', '#2ecc71', '#2ecc71', '#2ecc71', '#2ecc71', '#e74c3c']

bars = axes[0].bar(models, params, color=colors_bar, edgecolor='white', linewidth=1.5)
axes[0].set_yscale('log')
axes[0].set_ylabel('Parameters (Billions)', fontsize=12)
axes[0].set_title('Model Size Evolution', fontsize=14, fontweight='bold')
for bar, p in zip(bars, params):
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height() * 1.2,
                f'{p}B', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 2. 아키텍처 차원 비교
arch_data = {
    'Model': ['GPT-1', 'GPT-2 XL', 'GPT-3'],
    'Layers': [12, 48, 96],
    'd_model': [768, 1600, 12288],
    'Heads': [12, 25, 96],
    'Context': [512, 1024, 2048]
}

x = np.arange(len(arch_data['Model']))
width = 0.2

# 정규화하여 비교
for i, (metric, values) in enumerate([(k, v) for k, v in arch_data.items() if k != 'Model']):
    normalized = [v / max(values) for v in values]
    bars = axes[1].bar(x + i*width, normalized, width, label=metric, alpha=0.8)
    for bar, orig in zip(bars, values):
        axes[1].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.02,
                    str(orig), ha='center', va='bottom', fontsize=8, fontweight='bold')

axes[1].set_xticks(x + width * 1.5)
axes[1].set_xticklabels(arch_data['Model'])
axes[1].set_ylabel('Normalized Value', fontsize=12)
axes[1].set_title('Architecture Dimensions', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=9)

# 3. 학습 데이터 크기 비교
data_models = ['GPT-1\n(BooksCorpus)', 'GPT-2\n(WebText)', 'GPT-3\n(Mixed)']
data_sizes = [5, 40, 570]  # GB
data_colors = ['#3498db', '#2ecc71', '#e74c3c']

bars = axes[2].bar(data_models, data_sizes, color=data_colors, edgecolor='white')
axes[2].set_ylabel('Training Data Size (GB)', fontsize=12)
axes[2].set_title('Training Data Scale', fontsize=14, fontweight='bold')
for bar, s in zip(bars, data_sizes):
    axes[2].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 5,
                f'{s} GB', ha='center', va='bottom', fontweight='bold')

plt.suptitle('GPT Series: Scale Comparison', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## Part 7: 진화의 핵심을 요약하는 다이어그램

In [ ]:
# ============================================================
# GPT 시리즈 진화 타임라인 다이어그램
# ============================================================

fig, ax = plt.subplots(figsize=(20, 10))
ax.set_xlim(0, 30)
ax.set_ylim(0, 20)
ax.axis('off')

# 타이틀
ax.text(15, 19, 'Evolution of the GPT Architecture',
        fontsize=22, fontweight='bold', ha='center', va='center',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='#2C3E50', edgecolor='none'),
        color='white')

# GPT-1 Box
gpt1_box = mpatches.FancyBboxPatch((1, 10), 7, 7,
    boxstyle=mpatches.BoxStyle("Round", pad=0.3),
    facecolor='#3498db', edgecolor='white', linewidth=2, alpha=0.9)
ax.add_patch(gpt1_box)
ax.text(4.5, 16.2, 'GPT-1 (Jun 2018)', fontsize=14, fontweight='bold', ha='center', color='white')
ax.text(4.5, 15.2, '117M params', fontsize=11, ha='center', color='white')
gpt1_features = [
    '• Post-Norm (original Transformer)',
    '• 12 layers, d=768, 12 heads',
    '• Context: 512 tokens',
    '• BooksCorpus (~5GB)',
    '• Pre-train → Fine-tune',
    '• BPE vocab: 40,478'
]
for i, feat in enumerate(gpt1_features):
    ax.text(1.5, 14.2 - i * 0.7, feat, fontsize=9, color='white', family='monospace')

# Arrow 1
ax.annotate('', xy=(10, 13.5), xytext=(8.5, 13.5),
            arrowprops=dict(arrowstyle='->', lw=3, color='#E74C3C'))

# GPT-2 Box
gpt2_box = mpatches.FancyBboxPatch((10.5, 10), 8, 7,
    boxstyle=mpatches.BoxStyle("Round", pad=0.3),
    facecolor='#27AE60', edgecolor='white', linewidth=2, alpha=0.9)
ax.add_patch(gpt2_box)
ax.text(14.5, 16.2, 'GPT-2 (Feb 2019)', fontsize=14, fontweight='bold', ha='center', color='white')
ax.text(14.5, 15.2, '1.5B params (13x)', fontsize=11, ha='center', color='white')
gpt2_features = [
    '★ Pre-Norm LayerNorm',
    '★ Weight init: 1/√N scaling',
    '★ Final LayerNorm added',
    '• Context: 512 → 1024 tokens',
    '• WebText (~40GB)',
    '★ Zero-shot task transfer',
    '• Byte-level BPE: 50,257'
]
for i, feat in enumerate(gpt2_features):
    color = '#FFEB3B' if feat.startswith('★') else 'white'
    ax.text(11, 14.2 - i * 0.65, feat, fontsize=9, color=color, family='monospace')

# Arrow 2
ax.annotate('', xy=(20, 13.5), xytext=(19, 13.5),
            arrowprops=dict(arrowstyle='->', lw=3, color='#E74C3C'))

# GPT-3 Box
gpt3_box = mpatches.FancyBboxPatch((20.5, 10), 8.5, 7,
    boxstyle=mpatches.BoxStyle("Round", pad=0.3),
    facecolor='#8E44AD', edgecolor='white', linewidth=2, alpha=0.9)
ax.add_patch(gpt3_box)
ax.text(24.75, 16.2, 'GPT-3 (May 2020)', fontsize=14, fontweight='bold', ha='center', color='white')
ax.text(24.75, 15.2, '175B params (117x)', fontsize=11, ha='center', color='white')
gpt3_features = [
    '★ Alternating Dense+Sparse Attn',
    '• 96 layers, d=12288, 96 heads',
    '• Context: 1024 → 2048 tokens',
    '• 5 datasets (~570GB, 300B tok)',
    '★ Few-shot In-Context Learning',
    '★ Emergent abilities at scale'
]
for i, feat in enumerate(gpt3_features):
    color = '#FFEB3B' if feat.startswith('★') else 'white'
    ax.text(21, 14.2 - i * 0.7, feat, fontsize=9, color=color, family='monospace')

# 하단 범례
legend_box = mpatches.FancyBboxPatch((5, 1), 20, 3.5,
    boxstyle=mpatches.BoxStyle("Round", pad=0.3),
    facecolor='#ECF0F1', edgecolor='#BDC3C7', linewidth=1, alpha=0.9)
ax.add_patch(legend_box)
ax.text(15, 4, 'Key Paradigm Shifts (★ = New innovation)', fontsize=12, fontweight='bold', ha='center')
shifts = [
    'GPT-1→2: Post-Norm → Pre-Norm  |  Fine-tune → Zero-shot  |  1/√N weight scaling',
    'GPT-2→3: Dense → Alternating Sparse Attention  |  Zero-shot → Few-shot In-Context Learning'
]
for i, s in enumerate(shifts):
    ax.text(15, 2.8 - i * 1.0, s, fontsize=10, ha='center', family='monospace', color='#2C3E50')

plt.tight_layout()
plt.show()

---
## Part 8: 최종 정리 퀴즈

아래 질문들에 답하며 이번 랩에서 배운 내용을 정리해봅시다.

In [ ]:
# ============================================================
# 최종 퀴즈
# ============================================================

quiz_questions = [
    {
        'q': 'Q1. GPT-2에서 도입된 Pre-Norm의 수식은? (Post-Norm과 비교하여)',
        'choices': [
            'A) Output = LayerNorm(x + SubLayer(x))',
            'B) Output = x + SubLayer(LayerNorm(x))',
            'C) Output = LayerNorm(x) + SubLayer(x)',
            'D) Output = SubLayer(x + LayerNorm(x))'
        ],
        'answer': 'B'
    },
    {
        'q': 'Q2. GPT-2에서 residual layer의 weight를 초기화할 때 사용하는 scaling factor는?',
        'choices': [
            'A) 1/N (N = number of layers)',
            'B) 1/√N (N = number of residual connections)',
            'C) 1/√d_model',
            'D) 1/n_heads'
        ],
        'answer': 'B'
    },
    {
        'q': 'Q3. GPT-3의 attention 구조에서 새롭게 도입된 것은?',
        'choices': [
            'A) Multi-head attention',
            'B) Cross-attention',
            'C) Alternating dense and locally-banded sparse attention',
            'D) Grouped Query Attention (GQA)'
        ],
        'answer': 'C'
    },
    {
        'q': 'Q4. GPT 시리즈의 학습 패러다임 변화 순서는?',
        'choices': [
            'A) Zero-shot → Fine-tune → Few-shot',
            'B) Fine-tune → Few-shot → Zero-shot',
            'C) Fine-tune → Zero-shot → Few-shot (In-context Learning)',
            'D) Few-shot → Zero-shot → Fine-tune'
        ],
        'answer': 'C'
    },
    {
        'q': 'Q5. GPT-3의 학습 데이터에서 가장 큰 비율을 차지하는 데이터셋과 그 학습 가중치는?',
        'choices': [
            'A) Wikipedia, 60%',
            'B) Common Crawl, 60%',
            'C) WebText2, 60%',
            'D) Books1+Books2, 60%'
        ],
        'answer': 'B'
    },
    {
        'q': 'Q6. Pre-Norm이 Post-Norm보다 깊은 네트워크 학습에 유리한 이유는?',
        'choices': [
            'A) 파라미터 수가 줄어들기 때문',
            'B) 학습 속도가 빨라지기 때문',
            'C) Gradient flow가 안정화되어 vanishing/exploding gradient가 완화되기 때문',
            'D) Attention score가 더 정확해지기 때문'
        ],
        'answer': 'C'
    }
]

print("📝 최종 정리 퀴즈")
print("=" * 70)

score = 0
for i, quiz in enumerate(quiz_questions):
    print(f"\n{quiz['q']}")
    for c in quiz['choices']:
        print(f"  {c}")

    # 자동 채점 (학생이 직접 답할 수 있도록 수정 가능)
    print(f"  ✅ 정답: {quiz['answer']}")
    score += 1

print(f"\n{'=' * 70}")
print(f"점수: {score}/{len(quiz_questions)}")

---
## Part 9: 종합 요약 (Summary)

### GPT-1 → GPT-2: 아키텍처 혁신

| 변경사항 | 기술적 내용 | 효과 |
|---------|-----------|------|
| **Pre-Norm** | LayerNorm을 sublayer 입력 전으로 이동 | Gradient 안정화 → 더 깊은 네트워크 학습 가능 |
| **Weight Scaling** | Residual projection을 $1/\sqrt{N}$으로 초기화 | Activation variance 폭발 방지 |
| **Final LayerNorm** | 마지막 attention block 뒤에 추가 LayerNorm | 출력 안정화 |
| **Byte-level BPE** | 토크나이저를 Byte-level BPE로 변경 | OOV 문제 해결, 50,257 vocab |
| **Paradigm shift** | Fine-tune → Zero-shot | Task-specific 학습 불필요 |

### GPT-2 → GPT-3: 스케일링 + 패러다임 혁신

| 변경사항 | 기술적 내용 | 효과 |
|---------|-----------|------|
| **Sparse Attention** | Dense와 locally-banded sparse를 교대 사용 | 96 layers에서의 계산 효율 향상 |
| **Massive Scaling** | 117M → 175B (1,500x) | Emergent abilities 발현 |
| **In-Context Learning** | Prompt에 예시를 포함하여 task 수행 | Fine-tuning 없이 다양한 task 수행 |
| **Data Diversity** | 5개 소스의 가중 샘플링 | 고품질 데이터 우선 학습 |

### 핵심 교훈

1. **Scale matters, but architecture matters too**: 단순히 크기만 키운 것이 아니라 Pre-Norm, weight scaling 등의 아키텍처 변경이 대규모 학습을 가능하게 했습니다.

2. **Emergent abilities**: 충분한 스케일에서 Fine-tuning 없이도 task를 수행하는 능력(in-context learning)이 "창발"적으로 나타났습니다.

3. **Data quality > quantity**: GPT-3는 고품질 데이터를 더 자주 반복하고, 저품질 대규모 데이터는 다운샘플링하는 전략을 사용했습니다.

---

### 📚 참고 문헌 (References)

1. Radford, A., Narasimhan, K., Salimans, T., & Sutskever, I. (2018). *Improving Language Understanding by Generative Pre-Training*. OpenAI.

2. Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). *Language Models are Unsupervised Multitask Learners*. OpenAI.

3. Brown, T. B., et al. (2020). *Language Models are Few-Shot Learners*. NeurIPS 2020. arXiv:2005.14165.

4. Vaswani, A., et al. (2017). *Attention Is All You Need*. NeurIPS 2017. arXiv:1706.03762.

5. Child, R., Gray, S., Radford, A., & Sutskever, I. (2019). *Generating Long Sequences with Sparse Transformers*. arXiv:1904.10509.

6. Xiong, R., et al. (2020). *On Layer Normalization in the Transformer Architecture*. ICML 2020. arXiv:2002.04745. (Pre-Norm의 이론적 분석)